# Diffusion Models for Image Generation 

Generative image models (and generative AI as a whole) have gained significant attention over the last few years. Model types you may have heard of like GANs, have shown success in creating realistic looking images. Diffusion models are a relatively new class of models and have recently demonstrated state-of-the-art performance with image generation. Diffusion models were [first conceptualized for image generation in 2015](https://arxiv.org/pdf/1503.03585.pdf) and now underpin many of the leading image generation systems. Diffusion models consist of two processes:

* The forward (noising) process: Slowly add noise to an image, destroying structure in the data distribution 
* The reverse (de-noising) process: Learn the reverse diffusion process, yielding a generative model of the data

In other words, the forward process adds random noise to an image and the reverse process learns to remove noise from an image. After training, we can use the model from the reverse process to iteratively remove noise from a random-noise starting point and generate images.

In this lab, you will:
* Load the dataset 
* Define the de-noinsing model (U-net with residual connections) 
* Define the diffusion model (forward and reverse processes) 
* Train the diffusion model and visualize results 

### Setup 

In [ ]:
import math
import os

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

print(tf.__version__)

### Choose dataset
This lab can be ran with either the flowers dataset or mnist dataset. **NOTE**: To train the model on the flowers dataset, it requires a GPU. To train a model faster, or without a GPU, choose the mnist dataset.

In [ ]:
dataset_choice = "flowers"
# dataset_choice = 'mnist'

In [ ]:
if dataset_choice == "flowers":
    IMAGE_SIZE = 64
    IMAGE_CHANNELS = 3
    WIDTHS = [64, 128, 128, 256]
    DATASET_NAME = "oxford_flowers102"
    DATASET_SPLIT = "test"
    CENTRAL_CROP = 0.8

elif dataset_choice == "mnist":
    IMAGE_SIZE = 28
    IMAGE_CHANNELS = 1
    WIDTHS = [8, 16, 32]
    NUM_EXAMPLES = 10000

else:
    raise ValueError("Please specify dataset_choice in the cell above.")

BLOCK_DEPTH = 2
EPOCHS = 50
BATCH_SIZE = 32
MIN_SIGNAL_RATE = 0.02
MAX_SIGNAL_RATE = 0.95
PLOT_DIFFUSION_STEPS = 25
EMA = 0.999
OUTPUT_DIR = "diffusion_model"
CHECKPOINT_DIR = os.path.join(OUTPUT_DIR, "checkpoints")

print(f"Using dataset: {dataset_choice}")

### Load dataset 

Define a function to load the dataset. This lab uses either the [Oxford Flower Dataset](https://www.tensorflow.org/datasets/catalog/oxford_flowers102) or the MNIST dataset. The flowers dataset contains images of flowers commonly found in the UK, and the MNIST dataset contains grayscale images of handwritten digits. 

In [ ]:
# tf data ingest for flowers dataset
def _preprocess_flowers(data):
    image = data["image"]
    image = tf.image.central_crop(image, CENTRAL_CROP)
    image = tf.cast(image, tf.float32) / 255.0
    image = tf.image.resize(
        image, size=[IMAGE_SIZE, IMAGE_SIZE], antialias=True
    )
    return tf.clip_by_value(image, 0.0, 1.0)


def create_dataset_flowers(batch_size):
    ds = tfds.load(DATASET_NAME, split=DATASET_SPLIT)
    ds = ds.map(
        _preprocess_flowers, num_parallel_calls=tf.data.AUTOTUNE
    ).cache()
    return ds.batch(batch_size, drop_remainder=True).prefetch(tf.data.AUTOTUNE)


# tf data ingest for mnist dataset
def _preprocess_mnist(image):
    image = tf.cast(image, tf.float32) / 255.0
    image = tf.expand_dims(image, -1)
    return tf.clip_by_value(image, 0.0, 1.0)


def create_dataset_mnist(batch_size):
    (X_train, _), (_, _) = tf.keras.datasets.mnist.load_data()
    ds = tf.data.Dataset.from_tensor_slices(X_train[:NUM_EXAMPLES])
    ds = ds.map(_preprocess_mnist, num_parallel_calls=tf.data.AUTOTUNE).cache()
    return ds.batch(batch_size, drop_remainder=True)

### Forward Diffusion Process
We will have a diffusion process starting at time = 0, and ending at time = 1. At each diffusion time, we will have a noise level `noise_rate` and a signal level `signal_rate`. How do we know the noise and signal level for a given diffusion time? For this, we will use a simple cosine schedule as defined in `diffusion_schedule()`. We generate the noisy image (for the forward diffusion processes) by weighting random noise by the noise rate, and the training image by the signal rate, then adding them together.

**Note**: This code needs to be implemented in a training step of a subclasses model eventually. We are only running it here locally first to visualize the forward diffusion process. 

In [ ]:
def diffusion_schedule(
    diffusion_times, max_signal_rate=0.98, min_signal_rate=0.02
):
    # diffusion times -> angles
    start_angle = tf.acos(max_signal_rate)
    end_angle = tf.acos(min_signal_rate)

    diffusion_angles = start_angle + diffusion_times * (end_angle - start_angle)

    # angles -> signal and noise rates
    signal_rates = tf.cos(diffusion_angles)
    noise_rates = tf.sin(diffusion_angles)

    return noise_rates, signal_rates

### Visualize forward diffusion process
Apply this forward diffusion to a single image and visualize this forward diffusion of adding guassian noise at randomly sampled diffusion times between 0 and 1. 

In [ ]:
# Batch of data
if dataset_choice == "mnist":
    train_ds = create_dataset_mnist(BATCH_SIZE)
elif dataset_choice == "flowers":
    train_ds = create_dataset_flowers(BATCH_SIZE)

ds_iter = train_ds.as_numpy_iterator()
data_batch = ds_iter.__next__()

In [ ]:
# random noise the same size as input batch of images
noises = tf.random.normal(shape=data_batch.shape)

# random sample of diffusion times between 0 and 1
diffusion_times = tf.random.uniform(
    shape=(data_batch.shape[0], 1, 1, 1), minval=0.0, maxval=1.0
)

# compute noise and signal rates for each respective diffusion time
noise_rates, signal_rates = diffusion_schedule(diffusion_times)

# create noisy images
noisy_images = signal_rates * data_batch + noise_rates * noises

In [ ]:
# Visualize original images and noisy images
num_rows = 32
num_cols = 2
plt.figure(figsize=(8, 64))
index = 0
for row in range(num_rows - 1):
    plt.subplot(num_rows, num_cols, index + 1)
    plt.imshow(data_batch[row], cmap="gray")
    plt.title("Original Image")
    plt.axis("off")

    plt.subplot(num_rows, num_cols, index + 2)
    plt.imshow(noisy_images[row], cmap="gray")
    plt.title(
        f"t={tf.squeeze(diffusion_times[row]):.2f} | signal={tf.squeeze(signal_rates[row]):.2f} | noise={tf.squeeze(noise_rates[row]):.2f}"
    )
    plt.axis("off")
    index += 2

plt.tight_layout()
plt.show()

As you can see visually, a small diffusion time (high signal rate, low noise rate) only adds a little noise and we can still (even visually) see what the original image is. Larger diffusion times (low signal rate, high noise rate) diffuse more "aggresively" and we can visually see the structure of the initial image is essentially, completely gone. 

**NOTE** You can run the previous two cells again to visualize another forward diffusion process, since diffusion times are randomly sampled.  

### De-noinsing Model Architecture

Now we need to specify the neural network that will be used for denoising (predicting the noise added to a given image). U-Net is a popular semantic segmentation architecture, whose main idea is that it progressively downsamples and then upsamples its input image, and adds skip connections between layers having the same resolution.

Our U-Net takes two inputs: the noisy images and the variances of their noise components. The variances are required since denoising a signal requires different operations at different levels of noise. In other words, the model needs to learn how to denoise at different noise rates so we will embed noise variance in such a way that different noise variances are represented differently. We will transform the noise variances using sinusoidal embeddings. This is quite similar to positional encodings used in transformers. The output of the model is the predicted noise. 

In [ ]:
class SinEmbedding(tf.keras.layers.Layer):
    """
    Custom layer for embedding noise rate variance.
    """

    def __init__(
        self, embedding_dim=32, min_freq=1.0, max_freq=1000.0, **kwargs
    ):
        super().__init__(**kwargs)
        self.frequencies = tf.exp(
            tf.linspace(
                tf.math.log(min_freq), tf.math.log(max_freq), embedding_dim // 2
            )
        )

        self.angular_speeds = 2.0 * math.pi * self.frequencies

    def get_config(self):
        config = super().get_config()
        return config

    def call(self, inputs):
        inputs = tf.cast(inputs, dtype=tf.float32)
        emb = tf.concat(
            [
                tf.sin(self.angular_speeds * inputs),
                tf.cos(self.angular_speeds * inputs),
            ],
            axis=3,
        )
        return emb


def ResidualBlock(width):
    """
    Batch norm followed by two conv layers to specified width.
    Input also skips these layers and is added to output.
    """

    def apply(x):
        input_width = x.shape[3]
        if input_width == width:
            residual = x
        else:
            residual = tf.keras.layers.Conv2D(width, kernel_size=1)(x)
        x = tf.keras.layers.BatchNormalization(center=False, scale=False)(x)
        x = tf.keras.layers.Conv2D(
            width,
            kernel_size=3,
            padding="same",
            activation=tf.keras.activations.swish,
        )(x)
        x = tf.keras.layers.Conv2D(width, kernel_size=3, padding="same")(x)
        x = tf.keras.layers.Add()([x, residual])
        return x

    return apply


# Downsampling
def DownBlock(width, block_depth):
    """
    Component of down stack of residual U-Net. Stacks block_depth ResidualBlocks to specified
    width, followed by Average Pooling for dimensionality reduction i.e. downsampling.
    """

    def apply(x):
        x, skips = x
        for _ in range(block_depth):
            x = ResidualBlock(width)(x)
            skips.append(x)
        x = tf.keras.layers.AveragePooling2D(pool_size=2)(x)
        return x

    return apply


# Upsampling
def UpBlock(width, block_depth):
    """
    Component of up stack of residual U-Net. Applies bilinear upsampling and
    stacks block_depth ResidualBlocks to specified width. Concats skip connections
    from same resolution as downstack.
    """

    def apply(x):
        x, skips = x
        x = tf.keras.layers.UpSampling2D(size=2, interpolation="bilinear")(x)
        for _ in range(block_depth):
            x = tf.keras.layers.Concatenate()([x, skips.pop()])
            x = ResidualBlock(width)(x)
        return x

    return apply


def get_network(image_size, image_channels, widths, block_depth):
    noisy_images = tf.keras.Input(
        shape=(image_size, image_size, image_channels)
    )
    noise_variances = tf.keras.Input(shape=(1, 1, 1))

    e = SinEmbedding()(noise_variances)
    e = tf.keras.layers.UpSampling2D(size=image_size, interpolation="nearest")(
        e
    )

    x = tf.keras.layers.Conv2D(widths[0], kernel_size=1)(noisy_images)
    x = tf.keras.layers.Concatenate()([x, e])

    skips = []
    for width in widths[:-1]:
        x = DownBlock(width, block_depth)([x, skips])

    for _ in range(block_depth):
        x = ResidualBlock(widths[-1])(x)

    for width in reversed(widths[:-1]):
        x = UpBlock(width, block_depth)([x, skips])

    x = tf.keras.layers.Conv2D(
        image_channels, kernel_size=1, kernel_initializer="zeros"
    )(x)

    return tf.keras.Model(
        [noisy_images, noise_variances], x, name="residual_unet"
    )

### Diffusion Model

#### Training process
When subclassing `tf.keras.Model` the core logic of defining a training step is in `train_step()`. A single training step in our diffusion model does the following:
* Normalize the images and samples random Gaussian noise 
* Samples noise/signal rates from sampled diffusion times 
* Mixes the images with the noise according to noise/signal rates
* Sends the noisy images through the de-noising network (`denoise()`)
* Computes loss between predicted noise and noise 
* Updates model parameters 


#### Sampling/Generating (reverse diffusion)
When sampling or generating via reverse diffusion, the goal is the remove noise from an image using the trained model. At each step, it takes the previous estimate of the noisy image and separates it into image and noise using the trained network. Then it recombines these components using the signal and noise rate of the next step. This process occurs while iterating over diffusion times in reverse (i.e. 1 to 0). This logic is implemented in `reverse_diffusion()`. 



In [ ]:
class DiffusionModel(tf.keras.Model):
    def __init__(
        self,
        image_size,
        image_channels,
        widths,
        block_depth,
        batch_size,
        min_signal_rate,
        max_signal_rate,
        ema,
        plot_diffusion_steps,
    ):
        super().__init__()
        self.image_size = image_size
        self.image_channels = image_channels
        self.batch_size = batch_size
        self.min_signal_rate = min_signal_rate
        self.max_signal_rate = max_signal_rate
        self.plot_diffusion_steps = plot_diffusion_steps
        self.ema = ema

        self.normalizer = tf.keras.layers.Normalization()
        self.network = get_network(
            image_size, image_channels, widths, block_depth
        )
        self.ema_network = tf.keras.models.clone_model(self.network)

    def denormalize(self, images):
        # convert the pixel values back to 0-1 range
        images = self.normalizer.mean + images * self.normalizer.variance**0.5
        return tf.clip_by_value(images, 0.0, 1.0)

    def diffusion_schedule(self, diffusion_times):
        # diffusion times -> angles
        start_angle = tf.acos(self.max_signal_rate)
        end_angle = tf.acos(self.min_signal_rate)

        diffusion_angles = start_angle + diffusion_times * (
            end_angle - start_angle
        )

        # angles -> signal and noise rates
        signal_rates = tf.cos(diffusion_angles)
        noise_rates = tf.sin(diffusion_angles)

        return noise_rates, signal_rates

    def denoise(self, noisy_images, noise_rates, signal_rates, training):
        if training:
            network = self.network

        # Use EMA network for generation
        else:
            network = self.ema_network

        # predict noise component and calculate the image component using it
        pred_noises = network(
            [noisy_images, noise_rates**2], training=training
        )
        pred_images = (noisy_images - noise_rates * pred_noises) / signal_rates

        return pred_noises, pred_images

    def reverse_diffusion(self, initial_noise, diffusion_steps):
        # reverse diffusion = sampling
        num_images = initial_noise.shape[0]
        step_size = 1.0 / diffusion_steps

        # at the first sampling step, noisy_image is pure noise
        # signal rate is assumed to be min_signal_rate
        next_noisy_images = initial_noise
        for step in range(diffusion_steps):
            noisy_images = next_noisy_images

            # separate the current noisy image to its components
            diffusion_times = tf.ones((num_images, 1, 1, 1)) - step * step_size
            noise_rates, signal_rates = self.diffusion_schedule(diffusion_times)
            pred_noises, pred_images = self.denoise(
                noisy_images, noise_rates, signal_rates, training=False
            )
            # network used in eval mode

            # remix the predicted components using the next signal and noise rates
            next_diffusion_times = diffusion_times - step_size
            next_noise_rates, next_signal_rates = self.diffusion_schedule(
                next_diffusion_times
            )
            next_noisy_images = (
                next_signal_rates * pred_images + next_noise_rates * pred_noises
            )
            # this new noisy image will be used in the next step

        return pred_images

    def generate(self, num_images, diffusion_steps):
        # noise -> images -> denormalized images
        initial_noise = tf.random.normal(
            shape=(
                num_images,
                self.image_size,
                self.image_size,
                self.image_channels,
            )
        )
        generated_images = self.reverse_diffusion(
            initial_noise, diffusion_steps
        )
        generated_images = self.denormalize(generated_images)
        return generated_images

    def train_step(self, images):
        images = self.normalizer(images, training=True)
        noises = tf.random.normal(
            shape=(
                self.batch_size,
                self.image_size,
                self.image_size,
                self.image_channels,
            )
        )

        diffusion_times = tf.random.uniform(
            shape=(self.batch_size, 1, 1, 1), minval=0.0, maxval=1.0
        )
        noise_rates, signal_rates = self.diffusion_schedule(diffusion_times)
        noisy_images = signal_rates * images + noise_rates * noises

        with tf.GradientTape() as tape:
            pred_noises, pred_images = self.denoise(
                noisy_images, noise_rates, signal_rates, training=True
            )

            noise_loss = self.loss(noises, pred_noises)  # used for training

        gradients = tape.gradient(noise_loss, self.network.trainable_weights)
        self.optimizer.apply_gradients(
            zip(gradients, self.network.trainable_weights)
        )

        # track the exponential moving averages of weights
        for weight, ema_weight in zip(
            self.network.weights, self.ema_network.weights
        ):
            ema_weight.assign(self.ema * ema_weight + (1 - self.ema) * weight)

        return {"noise_loss": noise_loss}

    def plot_images(self, epoch=None, logs=None, num_rows=3, num_cols=6):
        # plot random generated images for visual evaluation of generation quality
        generated_images = self.generate(
            num_images=num_rows * num_cols,
            diffusion_steps=self.plot_diffusion_steps,
        )

        # plot in grayscale for single channel images
        cmap = None
        if self.image_channels == 1:
            cmap = "gray"

        plt.figure(figsize=(num_cols * 2.0, num_rows * 2.0))
        for row in range(num_rows):
            for col in range(num_cols):
                index = row * num_cols + col
                plt.subplot(num_rows, num_cols, index + 1)
                plt.imshow(generated_images[index], cmap=cmap)
                plt.axis("off")
        plt.tight_layout()
        plt.show()
        plt.close()

### Train the model 
Before training the model, you need to compile it and specify an optimizer and loss function. We will simply use Adam as an optimizer and mean squared error to compute the loss between the actual noise added to an image and the predicted noise. 

The model has callbacks that will generate and display images after each epoch of model training, and save checkpoints of the best model (lowest loss).  

In [ ]:
model = DiffusionModel(
    image_size=IMAGE_SIZE,
    image_channels=IMAGE_CHANNELS,
    widths=WIDTHS,
    block_depth=BLOCK_DEPTH,
    batch_size=BATCH_SIZE,
    min_signal_rate=MIN_SIGNAL_RATE,
    max_signal_rate=MAX_SIGNAL_RATE,
    plot_diffusion_steps=PLOT_DIFFUSION_STEPS,
    ema=EMA,
)

model.compile(optimizer="adam", loss=tf.keras.losses.MeanSquaredError())

# compute mean/variance of dataset for normalization layer
model.normalizer.adapt(train_ds)

model.fit(
    train_ds,
    epochs=EPOCHS,
    callbacks=[
        tf.keras.callbacks.LambdaCallback(on_epoch_end=model.plot_images),
        tf.keras.callbacks.ModelCheckpoint(
            CHECKPOINT_DIR,
            monitor="noise_loss",
            mode="min",
            save_best_only=True,
            save_weights_only=True,
        ),
    ],
)

Congrats! You've succesfully trained a diffusion model to generate images. 